In [4]:

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import openai
from openai import OpenAI

In [2]:
#Ubicamos el articulo principal del dia 

url  = "https://www.lanacion.com.ar/"
laNacion = webdriver.Chrome()
laNacion.get(url)

# Wait for some time to allow dynamic content to load (you may need to adjust this)
laNacion.implicitly_wait(10)

# Wait for some time to allow dynamic content to load
wait = WebDriverWait(laNacion, 10)

# Locate the first article title using the provided HTML snippet
first_article_title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.text.ln-text.title span.text.ln-text.lead')))
first_article_title.click()

try:
    # Espero la publicidad
    wait = WebDriverWait(laNacion, 10)

    # Intento ubicar el botton "cerrar"
    cerrar_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()='cerrar']")))

    # Clickeo "cerrar"
    cerrar_button.click()
    print("Publicidad cerrada")

except TimeoutException:
    #Pasaron 10s y no hubo publicidad
    print("No hubo publicidad")

finally:

    # Esperamos hasta que se cargue el articlo y guardamos el texto 
    wait = WebDriverWait(laNacion, 10)
    first_article = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section.cuerpo__nota')))
    text = first_article.text
    laNacion.quit()


print(text)




No hubo publicidad
115
ESCUCHAR
El plenario de comisiones de Presupuesto y Hacienda; Legislación General y Asuntos Constitucionales está reunido hoy y continúa con el debate de la ley “ómnibus”. El primer funcionario del Poder Ejecutivo que expone es el ministro del Interior, Guillermo Francos. A lo largo del debate, el foco de las críticas se posó sobre la modificación del modo de elegir a los diputados nacionales -el Gobierno propone adoptar un sistema de elección por circunscripciones uninominales-. Además, se cuestionó la necesidad y la urgencia de tratar esta reforma en las sesiones extraordinarias.
A pesar de las chicanas, el oficialismo dio señales, entre ayer y hoy, de estar dispuesto a dar el debate que necesitan las medidas que pretende impulsar. La oposición “de centro”, que busca dialogar con los libertarios, lo reconocen y celebran. “Están más conscientes de la dificultad de conseguir un dictamen de mayoría si no aceptan cambios”, reflexionan.
Por eso, el ministro del Inte

In [6]:
# Una vez que obtuvimos el texto del articulo, lo resumimos usando ChatGTP

client = OpenAI(api_key="tu llave de OpenAI")


completion = client.chat.completions.create(
  #Aca estamos usando el modelo mas barato, sale $0.001 USD y tenes $5 USD menseuales gratuitos
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": f"Haz un resumen del siguiente texto: {text}"}
  ]
)

#La noticia es del dia 1/10/2024
print(completion.choices[0].message)

ChatCompletionMessage(content='En el plenario de comisiones de Presupuesto y Hacienda, Legislación General y Asuntos Constitucionales se está debatiendo la ley "ómnibus". El ministro del Interior, Guillermo Francos, defendió las reformas propuestas por el Gobierno, pero admitió la posibilidad de realizar correcciones. El foco de las críticas se centra en la modificación del sistema de elección de los diputados nacionales y en la necesidad de tratar esta reforma en sesiones extraordinarias. El proyecto también busca derogar las PASO y modificar el financiamiento de las campañas y la difusión de los spots electorales.', role='assistant', function_call=None, tool_calls=None)
